In [ ]:
#Importation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


In [ ]:
base_donnees = pd.read_csv("Donnés\BaseDeDonneesPlusComplete1970-2025.csv",index_col = "date")
#creeer une nouvelle colonne pour predire contenant la prediction de s'il va neiger et pleuvoir.
base_donnees["It's raining"] = base_donnees["rain"]>0
base_donnees["It's snowing"] = base_donnees["snow"]>0
del base_donnees["sunset_hhmm"]
del base_donnees["sunrise_hhmm"]
base_donnees = base_donnees.iloc[::-1]
base_donnees


In [ ]:
#Voir les pourcentages de manque de donnes dans chaque colonnes
null_prc = base_donnees.isnull().sum()/base_donnees.shape[0]
null_prc

In [ ]:
#Trier les colonnes valide avec peu de manque de donnees.
colonne_valide = base_donnees.columns[null_prc<.05].copy()
colonne_valide.shape[0]

In [ ]:
#Creer nouvelle base de donnes nettoyée
base_donnees = base_donnees[colonne_valide]
base_donnees

In [ ]:
#Étape pour trouver le manque de données se situe ou.
null_prc = base_donnees.isnull().sum()/base_donnees.shape[0]
null_prc

In [ ]:
#Technique de remplissage impeu brute qui remplace les valeurs vide par la valeur de l'ancienne journée
base_donnees = base_donnees.ffill()
#Verification de manque de données aprés remplissage.
null_prc = base_donnees.isnull().sum()/base_donnees.shape[0]
null_prc

In [ ]:
base_donnees.info()
#Retire les espaces devant les titres des columns pour réduire l'espace utilisé
base_donnees.columns = base_donnees.columns.str.strip()

In [ ]:
base_donnees.info()

In [ ]:
#Transformer mes valeurs de True/False de it's raining & it's snowing pour 1/0.
base_donnees["It's raining"] = base_donnees["It's raining"].map({True:1,False:0})
base_donnees["It's snowing"] = base_donnees["It's snowing"].map({True:1,False:0})
base_donnees

In [ ]:
#Parametrer les graphes.
sns.set_style(style="whitegrid")
base_donnees.describe()

In [ ]:
base_donnees.dtypes
base_donnees.columns

In [ ]:
#Voir la comparaison entre le nombre de jour qu'il pleu/pleu pas et le nombre de jour ou il neige/il neige pas.

plt.figure(figsize=(6,4))

sns.countplot(x="It's raining",data = base_donnees)
plt.title("Repartition de la pluie")
plt.show()
sns.countplot(x="It's snowing",data = base_donnees)
plt.title("Repartition de la tombé de neige")
plt.show()





In [ ]:
#La corrélation de chaque parametre.
plt.figure(figsize=(30,24))
sns.heatmap(base_donnees.corr(),annot=True,cmap="coolwarm",fmt=".2f")
plt.title("Corrélation de chaque données")
plt.show()

Traitement des données

In [ ]:
base_donnees = base_donnees.drop(columns=['avg_hourly_temperature', 'avg_temperature','avg_hourly_dew_point','avg_dew_point','max_pressure_sea', 'avg_hourly_pressure_sea', 'avg_pressure_sea',
       'min_pressure_sea', 'max_pressure_station',
       'avg_hourly_pressure_station', 'avg_pressure_station',
       'min_pressure_station','growdegdays_5', 'growdegdays_7', 'growdegdays_10'])


In [ ]:
#Séparer les données en majoritaire et minoritaire de It's raining et It's snowing.
base_donnees_majoriatire_pluie = base_donnees[base_donnees["It's raining"]==0]
base_donnees_minoritaire_pluie = base_donnees[base_donnees["It's raining"]==1]

base_donnees_majoriatire_neige = base_donnees[base_donnees["It's snowing"]==0]
base_donnees_minoritaire_neige = base_donnees[base_donnees["It's snowing"]==1]


In [ ]:
print(base_donnees_majoriatire_pluie.shape)
print(base_donnees_minoritaire_pluie.shape)
print(base_donnees_majoriatire_neige.shape)
print(base_donnees_minoritaire_neige.shape)


In [ ]:
#On égalise le nombre de données minoritaire a nos données majoritaire.

base_donnees_majoritaire_pluie_diminuee = resample(base_donnees_majoriatire_pluie,replace=False,n_samples=len(base_donnees_minoritaire_pluie),random_state=42)
base_donnees_majoritaire_neige_diminuee = resample(base_donnees_majoriatire_neige,replace=False,n_samples=len(base_donnees_minoritaire_neige),random_state=42)


In [ ]:
base_donnees_diminuee_pluie = pd.concat([base_donnees_majoritaire_pluie_diminuee,base_donnees_minoritaire_pluie])
base_donnees_diminuee_neige = pd.concat([base_donnees_majoritaire_neige_diminuee,base_donnees_minoritaire_neige])
base_donnees_diminuee_neige.shape


In [ ]:
#Mélanger mes données pour pas que mon modèle n'ai d'ordre en mémoire.
base_donnees_diminuee_pluie = base_donnees_diminuee_pluie.sample(frac=1,random_state=42).reset_index(drop=True)
base_donnees_diminuee_neige = base_donnees_diminuee_neige.sample(frac=1,random_state=42).reset_index(drop=True)
base_donnees_diminuee_pluie.head()

In [ ]:
base_donnees_diminuee_pluie["It's raining"].value_counts()
base_donnees_diminuee_neige["It's snowing"].value_counts()


Création données de test et données pour l'entrainement

In [ ]:
X_pluie = base_donnees_diminuee_pluie.drop(columns="It's raining")
y_pluie = base_donnees_diminuee_pluie["It's raining"]
print(X_pluie)

X_neige = base_donnees_diminuee_neige.drop(columns="It's snowing")
y_neige = base_donnees_diminuee_pluie["It's snowing"]